In [1]:
!pip install torchgeo perceiver-pytorch evaluate datasets

  Using cached torchgeo-0.6.1-py3-none-any.whl.metadata (19 kB)
  Using cached perceiver_pytorch-0.8.8-py3-none-any.whl.metadata (672 bytes)
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached kornia-0.7.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached lightly-1.5.13-py3-none-any.whl.metadata (37 kB)
  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached Rtree-1.3.0-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached segmentation_models_pytorch-0.3.4-py3-none-any.whl.metadata (30 kB)
  Using cached torchmetrics-1.4.3-py3-none-any.whl.metadata (19 kB)
  Using cached kornia_rs-0.1.5-cp310-none-win_amd64.whl.metadata (8.9 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached lightly_utils-0.0.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached aenum-3.1.15-py3-n

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Prof.RaajRamsankaran\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\kornia\\losses\\cauchy.py'
Consider using the `--user` option or check the permissions.



In [1]:
import os
#os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchgeo.datasets import MillionAID
import torchvision.transforms as transforms
from transformers import PerceiverForImageClassificationLearned
from torch.utils.data import random_split
from transformers import AdamW
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import evaluate
from torch.optim.lr_scheduler import CosineAnnealingLR

In [10]:
# Define transformations for the data
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert to PIL Image
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert back to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

In [11]:
# Function to apply transform directly to the dataset
class CustomMillionAID(MillionAID):
    def __getitem__(self, index):
        data = super().__getitem__(index)
        data['image'] = transform(data['image'])
        return data

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("--------------",device)

-------------- cuda


In [13]:
#device='cpu'
allocated_memory=torch.cuda.memory_allocated()
print(allocated_memory)

0


In [14]:
# Loading the dataset
train_ds = CustomMillionAID(root='./data', task="multi-class", split="train")
# Define the split ratio (e.g., 80% for training, 20% for testing)
train_size = int(0.8 * len(train_ds))
test_size = len(train_ds) - train_size
# Split the dataset
train_dataset, test_dataset = random_split(train_ds, [train_size, test_size])
# Define DataLoader for the training and test sets
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)#, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=0)

In [15]:
for i, data in enumerate(trainloader,0):
    print (i, data)
    break

0 {'image': tensor([[[[-1.1247, -1.3302, -1.3987,  ..., -0.5424, -0.4568, -0.6109],
          [-1.5699, -1.3644, -1.1589,  ..., -0.4568, -0.4397, -0.5767],
          [-1.7240, -1.4672, -1.2103,  ..., -0.4397, -0.4739, -0.5596],
          ...,
          [ 0.1597,  0.1597,  0.1939,  ..., -0.0116, -0.0287, -0.0801],
          [ 0.1426,  0.1768,  0.1939,  ..., -0.0801, -0.1143, -0.1314],
          [ 0.0741,  0.1426,  0.1768,  ..., -0.1999, -0.2513, -0.1999]],

         [[-0.6352, -0.8452, -0.9153,  ..., -0.1099, -0.0049, -0.1450],
          [-1.1078, -0.8978, -0.6702,  ..., -0.0224,  0.0126, -0.1275],
          [-1.3004, -1.0378, -0.7752,  ..., -0.0049, -0.0399, -0.1275],
          ...,
          [ 0.1176,  0.1176,  0.1527,  ...,  0.0126, -0.0049, -0.0574],
          [ 0.1001,  0.1352,  0.1527,  ..., -0.0224, -0.0399, -0.0574],
          [ 0.0301,  0.1001,  0.1352,  ..., -0.1099, -0.1625, -0.1099]],

         [[-0.4450, -0.6541, -0.7238,  ...,  0.1476,  0.2348,  0.0953],
          [-0.8981

In [16]:
model = PerceiverForImageClassificationLearned.from_pretrained("deepmind/vision-perceiver-learned",
                                                               num_labels=51,
                                                               ignore_mismatched_sizes=True)
model.to(device)

Some weights of PerceiverForImageClassificationLearned were not initialized from the model checkpoint at deepmind/vision-perceiver-learned and are newly initialized because the shapes did not match:
- perceiver.decoder.decoder.final_layer.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([51, 1024]) in the model instantiated
- perceiver.decoder.decoder.final_layer.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([51]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PerceiverForImageClassificationLearned(
  (perceiver): PerceiverModel(
    (input_preprocessor): PerceiverImagePreprocessor(
      (convnet_1x1): Conv2d(3, 256, kernel_size=(1, 1), stride=(1, 1))
      (position_embeddings): PerceiverTrainablePositionEncoding()
      (positions_projection): Linear(in_features=256, out_features=256, bias=True)
      (conv_after_patches): Identity()
    )
    (embeddings): PerceiverEmbeddings()
    (encoder): PerceiverEncoder(
      (cross_attention): PerceiverLayer(
        (attention): PerceiverAttention(
          (self): PerceiverSelfAttention(
            (layernorm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (layernorm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (query): Linear(in_features=1024, out_features=512, bias=True)
            (key): Linear(in_features=512, out_features=512, bias=True)
            (value): Linear(in_features=512, out_features=512, bias=True)
            (dropout): Dropout(

In [17]:
# Directory to save the model
save_dir = "E:\danny_dl_models\saved_models"
os.makedirs(save_dir, exist_ok=True)

In [18]:
# Total number of training steps
total_steps = len(trainloader) * 10  # 10 epochs
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-6)

C:\Users\Prof.RaajRamsankaran\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
# Training loop
def train(model, train_loader, optimizer, device, num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    output_filename = 'G:\danny\saved_models_perceiver\logs.txt'

    with open(output_filename, 'w') as f:
        for epoch in range(num_epochs):
            print("Epoch:", epoch)
            #f.write('Epoch' + ' ' + epoch + '\n')
            for batch in tqdm(trainloader):
                # get the inputs;
                inputs = batch['image'].to(device)
                labels = batch['label'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs=inputs, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                # evaluate
                predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
                accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
                f.write(f'epoch {epoch} Loss  {loss.item()} Accuracy {accuracy} \n')
                f.flush()
                #print(f"Loss: {loss.item()}, Accuracy: {accuracy}")
            
            scheduler.step()
            # Save model after each epoch
            model_save_path = os.path.join(save_dir, f'model_epoch{epoch+1}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            }, model_save_path)
            print(f"Model saved after epoch {epoch+1}")

In [20]:
# Number of epochs
num_epochs = 10

In [21]:
train(model, trainloader, optimizer, device, num_epochs)

accuracy = evaluate.load("accuracy")

model.eval()
for batch in tqdm(testloader):
      # get the inputs; 
      inputs = batch["image"].to(device)
      labels = batch["label"].to(device)

      # forward pass
      outputs = model(inputs=inputs, labels=labels)
      logits = outputs.logits 
      predictions = logits.argmax(-1).cpu().detach().numpy()
      references = batch["label"].numpy()
      accuracy.add_batch(predictions=predictions, references=references)

final_score = accuracy.compute()
print("Accuracy on test set:", final_score)

Epoch: 0


  0%|          | 0/4000 [00:00<?, ?it/s]

C:\Users\Prof.RaajRamsankaran\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (110124036 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Model saved after epoch 1
Epoch: 1


  0%|          | 0/4000 [00:00<?, ?it/s]

...

C:\Users\Prof.RaajRamsankaran\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (110124036 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Model saved after epoch 10


  0%|          | 0/1000 [00:00<?, ?it/s]

C:\Users\Prof.RaajRamsankaran\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (93508900 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Accuracy on test set: {'accuracy': 0.8945}
